In [ ]:
# %pip install office365-rest-python-client

from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

import os

site_url = "https://smartdrivinglabs.sharepoint.com/sites/inteligencia_de_negocios"

ctx = ClientContext(site_url).with_credentials(UserCredential("Desenvolvedor@questarbr.com", "Dev@2025!"))

folder_url = "/sites/inteligencia_de_negocios/Documentos Compartilhados/02_Dados_Parquet"

folder = ctx.web.get_folder_by_server_relative_url(folder_url)
files = folder.files
ctx.load(files)
ctx.execute_query()

# Pasta local temporária
local_folder = "/lakehouse/default/Files/Arquivos_Parquet"

for file in files:
    file_name = file.properties["Name"]
    file_url = file.properties["ServerRelativeUrl"]

    # Fazer o download do arquivo
    file_response = File.open_binary(ctx, file_url)

    # Criar o caminho de destino no Lakehouse
    local_path = os.path.join(local_folder, file_name)

    # Salvar o arquivo no Lakehouse
    with open(local_path, "wb") as local_file:
        local_file.write(file_response.content)

    print(f"Arquivo {file_name} copiado para {local_path}")
